<a href="https://colab.research.google.com/github/michele-perrone/SpectrogramPlayer/blob/main/Source/GstPEAQ_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GstPEAQ testing**
In this notebook, we use GstPEAQ to test the results obtained by the various techniques.   
The audio clips must be available as `.wav` audio files.   
Please note that GstPEAQ comes only as Gst plugin or as a command-line utility. We will use it as the former by creating command strings to be executed on the local terminal. Therefore, you must install GstPEAQ on you local machine in order to run this notebook.   
**Important!** Make sure you set the correct `project_path`

In [17]:
# Import necessary libraries

import numpy as np
import pickle
import librosa
import pathlib
from pathlib import Path
import os
import subprocess
import soundfile as sf
import pickle as pkl

In [18]:
# Set the project path

project_path = pathlib.Path('/home/michele/Projects/SpectrogramPlayer')
%cd $project_path

/home/michele/Projects/SpectrogramPlayer


In [ ]:
# Test if GstPEAQ is available

output = subprocess.check_output("peaq --help", shell=True)
print(output.decode())

Usage:
  peaq [OPTION?] REFFILE TESTFILE

peaq computes the Objective Difference Grade based on ITU-R BS.1367-1 (but it
does not meet its conformance requirements).

Help Options:
  -h, --help                        Show help options
  --help-all                        Show all help options
  --help-gst                        Show GStreamer Options

Application Options:
  --version                         print version information
  --advanced                        use advanced version
  --basic                           use basic version (default)

Report bugs to: <martin.holters@hsu-hh.de>
GstPEAQ home page: <http://ant.hsu-hh.de/gstpeaq>



In [19]:
# Make a list of the paths where the different genres reside

speech_results_path = pathlib.Path('./Results/Speech')
speech_results_path = pathlib.Path(os.path.join(project_path, speech_results_path))

music_results_path = pathlib.Path('./Results/Music')
music_results_path = pathlib.Path(os.path.join(project_path, music_results_path))

urban_results_path = pathlib.Path('./Results/Urban')
urban_results_path = pathlib.Path(os.path.join(project_path, urban_results_path))

results_path_list = {'speech': speech_results_path, 
                     'music': music_results_path, 
                     'urban': urban_results_path}

In [20]:
num_of_test_files = 20

# For each genre... (music, speech, urban)
for current_genre in results_path_list: ## <---- FOR EACH GENRE BEGINS
  
  # Print the genre we're dealing with
  print("Currently processing: %s" %(current_genre))

  # Initialize an empty dictionary for PEAQ results
  peaq = {
      'no_phase': np.empty((num_of_test_files, 1), dtype=float),
      'griffin_lim': np.empty((num_of_test_files, 1), dtype=float),
      #'deepgl': np.empty((num_of_test_files, 1), dtype=float),
      'melgan': np.empty((num_of_test_files, 1), dtype=float),
      'hifigan': np.empty((num_of_test_files, 1), dtype=float),
      'uniglow': np.empty((num_of_test_files, 1), dtype=float),
      'squeezewave': np.empty((num_of_test_files, 1), dtype=float),
      'univnet': np.empty((num_of_test_files, 1), dtype=float),
  }

  # Import the pickle dictionary for this genre.
  # We're going to append to it the testing results of GSTPeaq (the `peaq` dictionary allocated above!)
  in_file = open(os.path.join(results_path_list[current_genre], 'metrics_' + current_genre + '.pkl'), "rb")
  metrics = pickle.load(in_file)
  in_file.close()

  # Create a list containing the paths to the reference signals.
  # The paths in the array are ordered from 0 onwards.
  signals_path = pathlib.Path(os.path.join(results_path_list[current_genre], "signals_" + current_genre))
  ref_signals_paths_list = []
  print("Looking for the reference signals...")
  for current_signal_path in sorted(signals_path.iterdir()): # Ordered in ascending order. Test signals will follow the same order!!
    ref_signals_paths_list.append(current_signal_path)  
    print("Found %s" %(current_signal_path.name))
  print("\n")

  # For each folder in the results path...
  # (each folder corresponds to a technique)
  for current_dir in results_path_list[current_genre].iterdir(): ## <---- FOR EACH TECHNIQUE BEGINS
    
    # ... check if it's a directory and that it's not the "signals_* folder"
    if current_dir.is_dir() and not str(current_dir.name).startswith("signals_"):
      current_technique = current_dir.name.replace('_' + current_genre,'')
      print("Current technique: %s" %(current_technique))
      print("Current folder: %s" %(current_dir))
      
      # Compare the reference signals with the reconstructed signals
      for current_rcstr_signal_path in sorted(current_dir.iterdir()): # Ordered in ascending order (same order as ref. signals!!)
        # For each reconstructed signal that we found, we go and look for the
        # reference signal in the "signals_*" folder.

        # Reference signal path:
        current_ref_signal_path = str(current_rcstr_signal_path).replace(str(current_technique), 'signals')
        print("Reference signal path: %s" %(current_ref_signal_path))
        # Reconstructed signal path:
        print("Reconstructed signal path: %s" %(current_rcstr_signal_path))
        
        # Compose the command string, used to invoke peaq from the console
        command = "peaq --basic %s %s" %(current_ref_signal_path, current_rcstr_signal_path)
        # Call peaq and store the output
        current_peaq_output = subprocess.check_output(command, shell=True).decode()
        # We have to un-format the output and keep only the Objective Difference Grade 
        current_peaq_output = subprocess.check_output(command, shell=True).decode().partition('\n')
        objective_difference_grade = float(current_peaq_output[0] # Keep only the first line
                                          .replace('Objective Difference Grade: ', '') # Remove the description
                                          ) # Convert the string to float [1,5]
        distortion_index = float(current_peaq_output[2] # Keep only the second line
                            .replace('Distortion Index: ', '') # Remove the description
                            )# Convert the string to float                                
        print("Objective difference grade: %.3f. Distortion index: %.3f\n" %(objective_difference_grade, distortion_index))
        # Store the ODG in the dictionary
        idx = int(current_rcstr_signal_path.name.replace('.wav', ''))
        peaq[current_technique][idx] = objective_difference_grade
        
    print("\n") ## <---- FOR EACH TECHNIQUE ENDS
  
  print("\n") ## <---- FOR EACH GENRE ENDS
  
  # Append the PEAQ results for this genre to the existing dictionary and save it to file
  metrics = {'RMSE': metrics['RMSE'],
            'NRMSE': metrics['NRMSE'],
            'PESQ': metrics['PESQ'],
            'MOSNET': metrics['MOSNET'],
            'RMSE_phase': metrics['RMSE_phase'],
            'NRMSE_phase': metrics['NRMSE_phase'],
            'PEAQ': peaq
            }

  out_file = open(os.path.join(results_path_list[current_genre], 'metrics_' + current_genre + '.pkl'), "wb")
  pickle.dump(metrics, out_file)
  out_file.close()

Currently processing: speech
Looking for the reference signals...
Found 0.wav
Found 1.wav
Found 10.wav
Found 11.wav
Found 12.wav
Found 13.wav
Found 14.wav
Found 15.wav
Found 16.wav
Found 17.wav
Found 18.wav
Found 19.wav
Found 2.wav
Found 20.wav
Found 3.wav
Found 4.wav
Found 5.wav
Found 6.wav
Found 7.wav
Found 8.wav
Found 9.wav


Current technique: no_phase
Current folder: /home/michele/Projects/SpectrogramPlayer/Results/Speech/no_phase_speech
Reference signal path: /home/michele/Projects/SpectrogramPlayer/Results/Speech/signals_speech/0.wav
Reconstructed signal path: /home/michele/Projects/SpectrogramPlayer/Results/Speech/no_phase_speech/0.wav
Objective difference grade: -3.913. Distortion index: -4.116

Reference signal path: /home/michele/Projects/SpectrogramPlayer/Results/Speech/signals_speech/1.wav
Reconstructed signal path: /home/michele/Projects/SpectrogramPlayer/Results/Speech/no_phase_speech/1.wav
Objective difference grade: -3.896. Distortion index: -3.888

Reference signal pa